In [1]:
import os
import glob
import xarray as xr
import numpy as np
import pandas as pd
import salem
from esmtools.grid import convert_lon
from utils import *

# options
xr.set_options(keep_attrs=True)

In [2]:
# Set the directory where the data is stored
dir_extremes = 'Trends_NC/Precip_trend/'
os.chdir(dir_extremes)

In [3]:
!dir

 O volume na unidade C nÆo tem nome.
 O N£mero de S‚rie do Volume ‚ FC9F-C5F6

 Pasta de C:\Users\Luiz\OneDrive\Climate_Extremes_DOC\Trends_NC\Precip_trend

06/05/2022  23:24    <DIR>          .
06/05/2022  23:24    <DIR>          ..
04/05/2022  12:42           229,145 cdd_Trend_SA.nc
04/05/2022  12:42           229,145 cwd_Trend_SA.nc
04/05/2022  12:42           229,145 prcptot_Trend_SA.nc
04/05/2022  12:43           229,145 r10mm_Trend_SA.nc
04/05/2022  12:42           229,145 r20mm_Trend_SA.nc
04/05/2022  12:42           229,145 r30mm_Trend_SA.nc
04/05/2022  12:42           229,145 r95p_Trend_SA.nc
04/05/2022  12:42           229,145 r99p_Trend_SA.nc
04/05/2022  12:42           229,145 rx1day_Trend_SA.nc
04/05/2022  12:42           229,145 rx5day_Trend_SA.nc
04/05/2022  12:42           229,145 sdii_Trend_SA.nc
04/05/2022  12:42         4,583,916 Trends-Precip.png
              12 arquivo(s)      7,104,511 bytes
               2 pasta(s)   57,839,206,400 bytes dispon¡veis


In [4]:
# Shape
shp = 'C:/Users/Luiz/Documents/Shapefiles/South_America_ecorreg_diss/South_America_ecorregions_dissolve.shp'
df_shp = salem.read_shapefile(shp)

In [5]:
# Get list of netcdf files
lista_nc = glob.glob('*.nc')
# Sort list by other list
list_index = ['r10mm', 'r20mm', 'r30mm', 'rx1day', 'rx5day', 'prcptot', 'sdii', 'r95p', 'r99p', 'cwd', 'cdd']
ETCCDI_index = [x for x in lista_nc if x.split('_')[0] in list_index]
ETCCDI_index.sort(key=lambda x: list_index.index(x.split('_')[0]))
ETCCDI_index

['r10mm_Trend_SA.nc',
 'r20mm_Trend_SA.nc',
 'r30mm_Trend_SA.nc',
 'rx1day_Trend_SA.nc',
 'rx5day_Trend_SA.nc',
 'prcptot_Trend_SA.nc',
 'sdii_Trend_SA.nc',
 'r95p_Trend_SA.nc',
 'r99p_Trend_SA.nc',
 'cwd_Trend_SA.nc',
 'cdd_Trend_SA.nc']

# Trends by Biomes

In [6]:
# get list to unique values biomes
biomes = list(df_shp.drop([5]).BIOME_NAME.unique())
biomes

['Deserts & Xeric Shrublands',
 'Flooded Grasslands & Savannas',
 'Mangroves',
 'Mediterranean Forests, Woodlands & Scrub',
 'Montane Grasslands & Shrublands',
 'Temperate Broadleaf & Mixed Forests',
 'Temperate Grasslands, Savannas & Shrublands',
 'Tropical & Subtropical Dry Broadleaf Forests',
 'Tropical & Subtropical Grasslands, Savannas & Shrublands',
 'Tropical & Subtropical Moist Broadleaf Forests',
 'Tundra']

In [53]:
alpha = 0.05
trend_list = []
sig_list = []
condition_list = []
biomes_list = []
index_list = []

for biome in biomes:
    for file_nc in ETCCDI_index:
        ds = xr.open_dataset(file_nc, decode_timedelta=False)
        # clip for each biome
        ds = mask_dataset(ds, shapefile_path=shp, col_shapefile='BIOME_NAME', query=biome)

        # Remove 'time_bnds' if exists
        if 'time_bnds' in ds:
            ds = ds.drop_vars('time_bnds')
        else:
            pass
        # Get values
        df = ds.to_dataframe()
        Trend = df['Trend']
        Sig = df['Sig']
        df['Biome'] = biome
        df['Index_ETCCDI'] = file_nc.split('_')[0]

        # Add categories values
        df.loc[(df.Sig <= alpha) & (df.Trend > 0), 'Condition'] = 'Positive Significant'
        df.loc[(df.Sig <= alpha) & (df.Trend < 0), 'Condition'] = 'Negative Significant'
        df.loc[(df.Sig > alpha) & (df.Trend > 0), 'Condition'] = 'Positive Non-Significant'
        df.loc[(df.Sig > alpha) & (df.Trend < 0), 'Condition'] = 'Negative Non-Significant'

        Condition = df['Condition']
        Biomes = df['Biome']
        Index = df['Index_ETCCDI']
        # Add to list
        trend_list.append(Trend)
        sig_list.append(Sig)
        condition_list.append(Condition)
        biomes_list.append(Biomes)
        index_list.append(Index)

        print('Get Values Pixel to Pixel for {}'.format(biome))
        print(f'Index: ', file_nc.upper().split('_')[0], ' Biome: ', biome)

print('Creating Dataframe...')
# Concatenate lists
a = pd.concat(trend_list)
b = pd.concat(sig_list)
c = pd.concat(condition_list)
d = pd.concat(biomes_list)
e = pd.concat(index_list)
# Create dataframe
df = pd.concat([a, b, c, d, e], axis=1).dropna()

print('Done!')

Get Values Pixel to Pixel for Deserts & Xeric Shrublands
Index:  R10MM  Biome:  Deserts & Xeric Shrublands
Get Values Pixel to Pixel for Deserts & Xeric Shrublands
Index:  R20MM  Biome:  Deserts & Xeric Shrublands
Get Values Pixel to Pixel for Deserts & Xeric Shrublands
Index:  R30MM  Biome:  Deserts & Xeric Shrublands
Get Values Pixel to Pixel for Deserts & Xeric Shrublands
Index:  RX1DAY  Biome:  Deserts & Xeric Shrublands
Get Values Pixel to Pixel for Deserts & Xeric Shrublands
Index:  RX5DAY  Biome:  Deserts & Xeric Shrublands
Get Values Pixel to Pixel for Deserts & Xeric Shrublands
Index:  PRCPTOT  Biome:  Deserts & Xeric Shrublands
Get Values Pixel to Pixel for Deserts & Xeric Shrublands
Index:  SDII  Biome:  Deserts & Xeric Shrublands
Get Values Pixel to Pixel for Deserts & Xeric Shrublands
Index:  R95P  Biome:  Deserts & Xeric Shrublands
Get Values Pixel to Pixel for Deserts & Xeric Shrublands
Index:  R99P  Biome:  Deserts & Xeric Shrublands
Get Values Pixel to Pixel for Desert

In [55]:
# Save dataframe
# df.to_excel('Trend_ETCCDI_Pixel_to_Pixel_Precip-Biomes.xlsx')

In [65]:
# Calculate percentage of Condition Trends for each biome
(df.groupby(['Biome', 'Index_ETCCDI'])['Condition'].value_counts(normalize=True) * 100).round(1).rename(
    'Percentage')

Biome                       Index_ETCCDI  Condition               
Deserts & Xeric Shrublands  cdd           Positive Non-Significant     41.5
                                          Negative Non-Significant     39.2
                                          Negative Significant         12.3
                                          Positive Significant          6.9
                            cwd           Negative Significant         45.5
                                                                      ...  
Tundra                      r20mm         Negative Significant        100.0
                            r95p          Negative Significant        100.0
                            rx1day        Negative Significant        100.0
                            rx5day        Negative Significant        100.0
                            sdii          Negative Significant        100.0
Name: Percentage, Length: 427, dtype: float64

# Trends by IPCC regions

In [7]:
shp_IPCC = 'C:/Users/Luiz/Documents/Shapefiles/IPCC_Regions_SA/IPCC_Regions_SA.shp'
df_shp_IPCC = salem.read_shapefile(shp_IPCC)

In [8]:
# Pegar lista com valores unicos
ipcc_regions = list(df_shp_IPCC.Acronym.unique())
ipcc_regions

['NWS', 'NSA', 'NES', 'SAM', 'SWS', 'SES', 'SSA']

In [20]:
alpha = 0.05
trend_list = []
sig_list = []
condition_list = []
IPCC_list = []
index_list = []

for region in ipcc_regions:
    for file_nc in ETCCDI_index:
        ds = xr.open_dataset(file_nc, decode_timedelta=False)
        # clip for each biome
        try:
            ds = mask_dataset(ds, shapefile_path=shp_IPCC, col_shapefile='Acronym', query=region)
        except IndexError as e:
            ds = ds.salem.subset(shape=df_shp_IPCC.query(f'Acronym == "{region}"'))
            ds = ds.salem.roi(shape=df_shp_IPCC.query(f'Acronym == "{region}"'))

        # Remove 'time_bnds' if exists
        if 'time_bnds' in ds:
            ds = ds.drop_vars('time_bnds')
        else:
            pass
        # Get values
        df = ds.to_dataframe()
        Trend = df['Trend']
        Sig = df['Sig']
        df['IPCC-Region'] = region
        df['Index_ETCCDI'] = file_nc.split('_')[0]

        # Add categories values
        df.loc[(df.Sig <= alpha) & (df.Trend > 0), 'Condition'] = 'Positive Significant'
        df.loc[(df.Sig <= alpha) & (df.Trend < 0), 'Condition'] = 'Negative Significant'
        df.loc[(df.Sig > alpha) & (df.Trend > 0), 'Condition'] = 'Positive Non-Significant'
        df.loc[(df.Sig > alpha) & (df.Trend < 0), 'Condition'] = 'Negative Non-Significant'

        Condition = df['Condition']
        IPCC_Regions = df['IPCC-Region']
        Index = df['Index_ETCCDI']
        # Add to list
        trend_list.append(Trend)
        sig_list.append(Sig)
        condition_list.append(Condition)
        IPCC_list.append(IPCC_Regions)
        index_list.append(Index)

        print('Get Values Pixel to Pixel for {}'.format(region))
        print(f'Index: ', file_nc.upper().split('_')[0], ' IPCC-Region: ', region)

print('Creating Dataframe...')
# Concatenate lists
a = pd.concat(trend_list)
b = pd.concat(sig_list)
c = pd.concat(condition_list)
d = pd.concat(IPCC_list)
e = pd.concat(index_list)
# Create dataframe
df = pd.concat([a, b, c, d, e], axis=1).dropna()

print('Done!')

Get Values Pixel to Pixel for NWS
Index:  R10MM  IPCC-Region:  NWS
Get Values Pixel to Pixel for NWS
Index:  R20MM  IPCC-Region:  NWS
Get Values Pixel to Pixel for NWS
Index:  R30MM  IPCC-Region:  NWS
Get Values Pixel to Pixel for NWS
Index:  RX1DAY  IPCC-Region:  NWS
Get Values Pixel to Pixel for NWS
Index:  RX5DAY  IPCC-Region:  NWS
Get Values Pixel to Pixel for NWS
Index:  PRCPTOT  IPCC-Region:  NWS
Get Values Pixel to Pixel for NWS
Index:  SDII  IPCC-Region:  NWS
Get Values Pixel to Pixel for NWS
Index:  R95P  IPCC-Region:  NWS
Get Values Pixel to Pixel for NWS
Index:  R99P  IPCC-Region:  NWS
Get Values Pixel to Pixel for NWS
Index:  CWD  IPCC-Region:  NWS
Get Values Pixel to Pixel for NWS
Index:  CDD  IPCC-Region:  NWS
Get Values Pixel to Pixel for NSA
Index:  R10MM  IPCC-Region:  NSA
Get Values Pixel to Pixel for NSA
Index:  R20MM  IPCC-Region:  NSA
Get Values Pixel to Pixel for NSA
Index:  R30MM  IPCC-Region:  NSA
Get Values Pixel to Pixel for NSA
Index:  RX1DAY  IPCC-Region:  N

In [22]:
# Save dataframe
df.to_excel('Trend_ETCCDI_Pixel_to_Pixel_Precip-IPCC-Regions.xlsx')

In [24]:
# Calculate percentage of Condition Trends for each IPCC region
(df.groupby(['IPCC-Region', 'Index_ETCCDI'])['Condition'].value_counts(normalize=True) * 100).round(1).rename(
    'Percentage')

IPCC-Region  Index_ETCCDI  Condition               
NES          cdd           Positive Non-Significant    49.6
                           Positive Significant        23.6
                           Negative Non-Significant    21.7
                           Negative Significant         5.1
             cwd           Negative Significant        90.5
                                                       ... 
SWS          rx5day        Positive Significant         6.4
             sdii          Positive Non-Significant    33.3
                           Negative Non-Significant    32.5
                           Negative Significant        18.7
                           Positive Significant        15.5
Name: Percentage, Length: 307, dtype: float64